In [80]:
import tensorflow as tf
import gym
import numpy as np
import datetime
from tensorflow_probability import distributions as tfpd, layers as tfpl

tf.random.set_seed(0)
tf.keras.backend.set_floatx('float32')

In [17]:
from tensorflow import keras
from tensorflow.keras import layers as kl

In [81]:
class GaussianSample(kl.Layer):
    def __init__(self, act_dim):
        super(GaussianSample, self).__init__()
        self.log_std = self.add_weight(
            'log_std', initializer=tf.keras.initializers.Constant(-0.53), 
            shape=(act_dim,), trainable=True
        )
        self.normal_dist = tfpl.DistributionLambda(
            make_distribution_fn=lambda t: tfpd.Normal(loc=t, scale=tf.exp(self.log_std)),
            convert_to_tensor_fn=lambda s: s.sample(),
        )
    
    #def build(self, input_shape):
    #    """
    #    input_shape: might be [None, act_dim]
    #    """
    #    self.log_std = self.add_weight(
    #        'log_std', initializer=tf.keras.initializers.Constant(-0.53), 
    #        shape=(input_shape[1],), dtype='float32', trainable=True
    #    )

    def call(self, input):
        #return tfpd.Normal(loc=input, scale=tf.exp(self.log_std))
        return self.normal_dist(input)

In [82]:
def get_actor(obs_dim: int, act_dim: int):
    obs = keras.Input(shape=(obs_dim,), name='observations')
    x = kl.Dense(32, activation='tanh', name='dense_1')(obs)
    x = kl.Dense(act_dim, name='logits')(x)

    pi = GaussianSample(act_dim)(x)
    
    model = keras.Model(inputs=obs,
                        outputs=[pi, x])
    
    return model

# Test model

In [64]:
test_model = get_actor(3, 1)

The following Variables were used a Lambda layer's call (distribution_lambda_4411), but
are not present in its tracked objects:
  <tf.Variable 'log_std:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


In [65]:
obs = np.array([[ 0.69950885,  0.6432279 , -7.588761  ]])
res = test_model(obs)
print(res[0].sample()[0].numpy())


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[-0.58579165]


In [83]:
def get_opt_fn(model, lr=3e-4):
    opt = tf.keras.optimizers.Adam(lr)
    
    #@tf.function
    def step_fn(obs_no, act_na, adv_n):
        with tf.GradientTape() as tape:
            logp = tf.reduce_sum(
                model(obs_no)[0].log_prob(act_na),
                axis=1,
            )
            
            loss = tf.reduce_mean(-logp * adv_n)
        
        grad = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(grad, model.trainable_variables))
        
        return loss
        
    return opt, step_fn

In [84]:
def reward_to_go(rews):
    n = len(rews)
    rtgs = np.zeros_like(rews)
    for i in reversed(range(n)):
        rtgs[i] = rews[i] + (rtgs[i+1] if i+1 < n else 0)
    return rtgs

In [87]:
def train(env, epochs=200, buffer_size=4096):
    assert isinstance(env.observation_space, gym.spaces.Box), \
        "This is only for continuous observation space"
    assert isinstance(env.action_space, gym.spaces.Box), \
        "This is only for continuous action space"
    
    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]
    
    model = get_actor(obs_dim, act_dim)
    model.summary()
    # keras.utils.plot_model(model, 'multi_output_model.png')
    
    
    opt, step_fn = get_opt_fn(model)
    
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for reward-to-go weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # collect experience by acting in the environment with current policy
        while True:
            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = model(obs[None])[0].sample()[0]
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a_t|s_t) is reward-to-go from t
                batch_weights += list(reward_to_go(ep_rews))

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # end experience loop if we have enough of it
                if len(batch_obs) > buffer_size:
                    break

        # take a single policy gradient update step
        batch_loss = step_fn(np.array(batch_obs), np.array(batch_acts), np.array(batch_weights))
        return batch_rets, batch_lens, batch_loss
    
    for i in range(epochs):
        batch_rets, batch_lens, batch_loss = train_one_epoch()
        if epochs % 50 == 0 or i == epochs-1:
            model.save_weights('ckpts/_model_'+str(i), save_format='tf')
        
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
              (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

# Training

In [88]:
env = gym.make('Pendulum-v0')
obs_dim = env.observation_space.shape[0]
obs_dim
act_dim = env.action_space.shape[0]
act_dim
train(env)

The following Variables were used a Lambda layer's call (distribution_lambda_4418), but
are not present in its tracked objects:
  <tf.Variable 'log_std:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
observations (InputLayer)    [(None, 3)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                128       
_________________________________________________________________
logits (Dense)               (None, 1)                 33        
_________________________________________________________________
gaussian_sample_20 (Gaussian (None, 1)                 1         
Total para

act_na (4200, 1)
epoch:  62 	 loss: -540.857 	 return: -1193.905 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  63 	 loss: -530.464 	 return: -1190.973 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  64 	 loss: -526.222 	 return: -1218.800 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  65 	 loss: -546.300 	 return: -1246.581 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  66 	 loss: -538.859 	 return: -1223.026 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  67 	 loss: -547.730 	 return: -1229.270 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  68 	 loss: -541.904 	 return: -1194.556 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  69 	 loss: -536.896 	 return: -1192.802 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  70 	 loss: -543.042 	 return: -1182.757 	 ep_len: 200.000
obs_no (4200, 3)
act_na (4200, 1)
epoch:  71 	 loss: -559.028 	 return: -1219.978 	 ep_len: 200.000
obs_no (4200, 3)


KeyboardInterrupt: 

# Evaluation

In [ ]:
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def load_saved_model(path, obs_dim, act_dim):
    model = get_actor(obs_dim, act_dim)
    model.load_weights(path)
    return model

In [ ]:
def evaluate_model(model, env):
    assert isinstance(env.observation_space, Box), \
        "This is only for continuous observation space"
    assert isinstance(env.action_space, Box), \
        "This is only for continuous action space"

    env.reset()
    img = plt.imshow(env.render(mode='rgb_array')) # only call this once

    for _ in range(100):
        img.set_data(env.render(mode='rgb_array')) # just update the data
        display.display(plt.gcf())
        display.clear_output(wait=True)
        action = model(obs[None])[1][0]
        env.step(action)

In [1]:

    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]

NameError: name 'env' is not defined

In [9]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        print(input_shape)
        self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

    def call(self, input):
        return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [10]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

(10, 5)


In [11]:
print([var.shape for var in layer.trainable_variables])

[TensorShape([5, 10])]
